In [ ]:
import cv2, dlib
import numpy as np
import math, sys
from dataPath import DATA_PATH
from dataPath import MODEL_PATH
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import matplotlib
matplotlib.rcParams['figure.figsize'] = (20.0, 20.0)
matplotlib.rcParams['image.cmap'] = 'gray'

In [ ]:
PREDICTOR_PATH = MODEL_PATH + "shape_predictor_68_face_landmarks.dat"
RESIZE_HEIGHT = 480
NUM_FRAMES_FOR_FPS = 100
SKIP_FRAMES = 1

In [ ]:
def interEyeDistance(predict):
    leftEyeLeftCorner = (predict[36].x, predict[36].y)
    rightEyeRightCorner = (predict[45].x, predict[45].y)
    distance = cv2.norm(np.array(rightEyeRightCorner) - np.array(leftEyeLeftCorner))
    distance = int(distance)
    return distance

In [ ]:
winName = "Stabilized facial landmark detector"
cv2.namedWindow(winName, cv2.WINDOW_NORMAL)

In [ ]:
videoFileName = ""

cap = cv2.VideoCapture(0)
if(cap.isOpened() == False):
    print("Unable to load video")

In [ ]:
winSize = 101
maxLevel = 10
fps = 30.0
ret, imPrev = cap.read()

In [ ]:
imGrayPrev = cv2.cvtColor(imPrev, cv2.COLOR_BGR2GRAY)

In [ ]:
size = imPrev.shape[0:1]
print(size)

In [ ]:
detector = dlib.get_frontal_face_detector()
landmarkDetector = dlib.shape_predictor(PREDICTOR_PATH)

In [ ]:
points = []
pointsPrev = []
pointsDetectedCur = []
pointsDetectedPrev = []

In [ ]:
eyeDistanceNotCalculated = True
eyeDistance = 0
isFirstFrame = True
fps = 10
showStabilized = False
count = 0

In [ ]:
# Summarize - 4 things to do in this
# 1 - detect the face at lower resolution and getting the bounding box, resize it at highest resolution
# 2 - 2 estimates of the location of the landmarks, one using detection we will run the facial landmark 
# detector on current frame to get an estimate and get estimate for facial landmark 
# also using the optical flow on the 
# current frame
# 3 - We need to combine these two estimates using alpha, linear combination of these 2 estimates and alpha
# is based on how fast the object is moving, fast moving obj - more weight to facial landmark detector,
# if the object is not fast moving - more weight to optical flow estimate
# 4 -

while(True):
    if (count == 0):
        t = cv2.getTickCount()
    
    ret, im = cap.read()
    imDlib = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    imGray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    height = im.shape[0]
    IMAGE_RESIZE = float(height)/RESIZE_HEIGHT
    
    imSmall = cv2.resize(im, None, fx=1.0/IMAGE_RESIZE, fy=1.0/IMAGE_RESIZE, interpolation = cv2.INTER_LINEAR)
    
    imSmallDlib = cv2.cvtColor(imSmall, cv2.COLOR_BGR2RGB)

    if (count % SKIP_FRAMES == 0):
        faces = detector(imSmallDlib,0)
        
    if len(faces) == 0 :
        print("No face detected")
        
    else:
        for i in range(0, len(faces)):
            print("Face detected")
            newRect = dlib.rectangle(int(faces[i].left() * IMAGE_RESIZE),
                                    int(faces[i].top() * IMAGE_RESIZE),
                                    int(faces[i].right() * IMAGE_RESIZE),
                                    int(faces[i].bottom() * IMAGE_RESIZE))
            
            landmarks = landmarkDetector(imDlib, newRect).parts()
            
            
        if(isFirstFrame == True):
            pointsPrev = []
            pointsDetectedPrev = []
            [pointsPrev.append((p.x, p.y)) for p in landmarks]
            [pointsDetectedPrev.append((p.x, p.y)) for p in landmarks]
            
        else:
            pointsPrev = [] # result of optical flow
            pointsDetectedPrev = [] # result of landmark detecttion in previous frame
            pointsPrev = points
            pointsDetectedPrev = pointsDetectedCur
            
        points = [] # points that we are going to output after stabilization
        pointsDetectedCur = []
        [points.append((p.x, p.y)) for p in landmarks] # this is going to change as we will modify it using optical flow
        [pointsDetectedCur.append((p.x, p.y)) for p in landmarks]
        
        
        # convert to numpy float array
        pointsArr = np.array(points, np.float32)
        pointsPrevArr = np.array(pointsPrev, np.float32)
        
        # if eye distance is not calculated before
        if eyeDistanceNotCalculated:
            eyeDistance = interEyeDistance(landmarks)
            print(eyeDistance)
            eyeDistanceNotCalculated = False
            
        if eyeDistance > 100:
            dotRadius = 3
        else:
            dotRadius = 2
            
        print(eyeDistance)
        
        sigma = eyeDistance * eyeDistance / 400
        
        s = 2 * int(eyeDistance/4)+1
        
        lk_params = dict(winSize = (s,s), maxLevel = 5, criteria = (cv2.TERM_CRITERIA_COUNT | cv2.TERM_CRITERIA_EPS, 20, 0.03))
        
        pointsArr, status, err = cv2.calcOpticalFlowPyrLK(imGrayPrev, imGray, pointsPrevArr, pointsArr, **lk_params)
        
        pointsArrFloat = np.array(pointsArr, np.float32)
        
        points = pointsArrFloat.tolist()
        
        for k in range(0, len(landmarks)):
            d = cv2.norm(np.array(pointsDetectedPrev[k]) - np.array(pointsDetectedCur[k]))
            alpha = math.exp(-d*d/sigma)
            points[k] = (1 - alpha) * np.array(pointsDetectedCur[k]) + alpha * np.array(points[k])
            
            
        if showStabilized is True:
            for p in points:
                cv2.circle(im, (int(p[0]), int(p[1])), dotRadius, (255, 0, 0), -1)
                
        else:
            for p in pointsDetectedCur:
                cv2.circle(im ,(int(p[0]), int(p[1])), dotRadius, (0, 0, 255), -1)
        
        isFirstFrame = False
        count = count + 1
        
        if (count == NUM_FRAMES_FOR_FPS):
            t = (cv2.getTickCount() - t )/ cv2.getTickFrequency()
            fps = NUM_FRAMES_FOR_FPS/t
            count = 0
            isFirstFrame = True
            
        cv2.putText(im, "{:.1f}-fps Stabilized = {!s}".format(fps, showStabilized), (50, size[0]-50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 3, cv2.LINE_AA)
        cv2.imshow(winName, im)
        key = cv2.waitKey(25) & 0xFF
        
        # use spacebar to toggle between stabilized and unstabilized version
        if key == 32:
            showStabilized = not showStabilized
        
        if key == 27:
            cv2.destroyAllWindows()
            cap.release()
            sys.exit()
        
        imPrev = im
        imGrayPrev = imGray
        
cv2.destroyAllWindows()
cap.release()
        